In [31]:
import pandas as pd
import numpy as np
from datetime import datetime
import tqdm as tqdm

In [32]:
match_level = pd.read_csv('data/664389efa0868_match_level_scorecard.csv')
batting_level = pd.read_csv('data/663e2b548c98c_batsman_level_scorecard.csv')
bowler_level = pd.read_csv('data/663e2b2c60743_bowler_level_scorecard.csv')
train_level = pd.read_csv('data/663e2b6d54457_train_data_with_samplefeatures.csv')
test_level = pd.read_csv('data/6644a1e287df6_test_data_with_samplefeatures.csv')

In [33]:
# converting team name to team ID in toos won

In [34]:
match_ids = match_level['match id'].tolist()
team1_ids = match_level['team1_id'].tolist()
team2_ids = match_level['team2_id'].tolist()
team1_names = match_level['team1'].tolist()
team2_names = match_level['team2'].tolist()
inning1_runs = match_level['inning1_runs'].tolist()
inning2_runs = match_level['inning2_runs'].tolist()
inning1_balls = match_level['inning1_balls'].tolist()
inning2_balls = match_level['inning2_balls'].tolist()
toss_decisions = match_level['toss decision'].tolist()
toss_winners = match_level['toss winner'].tolist()

inning1_wickets = match_level['inning1_wickets'].tolist()
inning2_wickets = match_level['inning2_wickets'].tolist()

match_date = match_level['match_dt'].tolist()


Batting strength


In [35]:
def scoreClassifier(score):
    """
    this function is used to calculate number of 30s, 50s and 100s.
    returns -> num30,num50,num100
    """
    num100 = 0
    num50 = 0
    num30 = 0
    while score>=100:
        num100 += 1
        score-=100
    while score>=50:
        num50+=1
        score-=50
    while score>=30:
        num30+=1
        score-=30
    
    return 0.2*num100 + 0.4*num50 + 0.4*num30

In [36]:
print(scoreClassifier(180))

1.0


In [37]:
def bat_avg(team1_id, team2_id, match_dt):
    # # Find the index of the input match ID
    # match_index = match_ids.index(match_id_input)
    # team1_id = team1_ids[match_index]
    # team2_id = team2_ids[match_index]
    # match_dt = match_date[match_index]
    match_dt = datetime.strptime(match_dt, '%Y-%m-%d')

    # Initialize variables to calculate batting averages
    total_runs_team1 = 0
    total_runs_team2 = 0
    matches_team1 = 0
    matches_team2 = 0
    strike_rate_sum_team1 = 0
    strike_rate_sum_team2 = 0
    scores_team1 = 0
    scores_team2 = 0

    # Iterate through all matches to calculate batting averages
    for i in range(len(match_ids)):
        if match_dt > datetime.strptime(match_date[i], '%Y-%m-%d'):
            if team1_ids[i] == team1_id or team2_ids[i] == team1_id:
                matches_team1 += 1
                if toss_winners[i] == team1_names[i]:
                    if toss_decisions[i] == 'bat':
                        total_runs_team1 += inning1_runs[i]
                        strike_rate_sum_team1 += inning1_runs[i]*100 / inning1_balls[i]
                        scores_team1 += scoreClassifier(inning1_runs[i])
                    else:
                        total_runs_team1 += inning2_runs[i]
                        strike_rate_sum_team1 += inning2_runs[i]*100 / inning2_balls[i]
                        scores_team1 += scoreClassifier(inning2_runs[i])
                
                else:
                    if toss_decisions[i] == 'bat':
                        total_runs_team1 += inning2_runs[i]
                        strike_rate_sum_team1 += inning2_runs[i]*100 / inning2_balls[i]
                        scores_team1 += scoreClassifier(inning2_runs[i])

                    else:
                        total_runs_team1 += inning1_runs[i]
                        strike_rate_sum_team1 += inning1_runs[i]*100 / inning1_balls[i]
                        scores_team1 += scoreClassifier(inning1_runs[i])
            
            if team1_ids[i] == team2_id or team2_ids[i] == team2_id:
                matches_team2 += 1
                if toss_winners[i] == team2_names[i]:
                    if toss_decisions[i] == 'bat':
                        total_runs_team2 += inning1_runs[i]
                        strike_rate_sum_team2 += inning1_runs[i]*100 / inning1_balls[i]
                        scores_team2 += scoreClassifier(inning1_runs[i])
                    else:
                        total_runs_team2 += inning2_runs[i]
                        strike_rate_sum_team2 += inning2_runs[i]*100 / inning2_balls[i]
                        scores_team2 += scoreClassifier(inning2_runs[i])
                else:
                    if toss_decisions[i] == 'bat':
                        total_runs_team2 += inning2_runs[i]
                        strike_rate_sum_team2 += inning2_runs[i]*100 / inning2_balls[i]
                        scores_team2 += scoreClassifier(inning2_runs[i])
                    else:
                        total_runs_team2 += inning1_runs[i]
                        strike_rate_sum_team2 += inning1_runs[i]*100 / inning1_balls[i]
                        scores_team2 += scoreClassifier(inning1_runs[i])

    
    bat_avg_team1 = total_runs_team1 / matches_team1 if matches_team1 != 0 else 0
    bat_avg_team2 = total_runs_team2 / matches_team2 if matches_team2 != 0 else 0
    strike_rate_avg_team1 = strike_rate_sum_team1/ matches_team1 if matches_team1 != 0 else 0
    strike_rate_avg_team2 = strike_rate_sum_team2/ matches_team2 if matches_team2 != 0 else 0

    return [bat_avg_team1, bat_avg_team2, strike_rate_avg_team1, strike_rate_avg_team2, scores_team1, scores_team2]


match_id_input = 9433696

bowl_avg_team1, bowl_avg_team2, strike_rate_team1, strike_rate_team2 , scores_team1, scores_team2 = bat_avg(11283, 12634, "2022-10-20")

print(f"Batting average for Team 1: {bowl_avg_team1}")
print(f"Batting average for Team 2: {bowl_avg_team2}")

print(f"Strike rate team 1: {strike_rate_team1}")
print(f"Strike rate team 2: {strike_rate_team2}")

print(f"scores team 1: {scores_team1}")
print(f"scores team 2: {scores_team2}")

Batting average for Team 1: 139.0
Batting average for Team 2: 132.27272727272728
Strike rate team 1: 113.78780198141655
Strike rate team 2: 114.16539392782994
scores team 1: 8.8
scores team 2: 5.0


In [38]:
def partnership(team1_id, team2_id, match_dt):
    
    # # Find the index of the input match ID
    # match_index = match_ids.index(match_id_input)
    # team1_id = team1_ids[match_index]
    # team2_id = team2_ids[match_index]

    bat_partnership_team1=0
    bat_partnership_team2=0
    matches_team1 = 0
    matches_team2 = 0
    match_dt = datetime.strptime(match_dt, '%Y-%m-%d')

    for i in range(len(match_ids)):
        if match_dt > datetime.strptime(match_date[i], '%Y-%m-%d'):
            if team1_ids[i] == team1_id or team2_ids[i] == team1_id:
                matches_team1 += 1
                if team1_names[i] == toss_winners[i]:
                    if toss_decisions[i] == 'bat':
                        
                        bat_partnership_team1+=inning1_runs[i]/(inning1_wickets[i]) if inning1_wickets[i]!=0 else inning1_runs[i]
                        bat_partnership_team2+=inning2_runs[i]/(inning2_wickets[i]) if inning2_wickets[i]!=0 else inning2_runs[i]
                    else:
                        bat_partnership_team1+=inning2_runs[i]/(inning2_wickets[i]) if inning2_wickets[i]!=0 else inning2_runs[i]
                        bat_partnership_team2+=inning1_runs[i]/(inning1_wickets[i]) if inning1_wickets[i]!=0 else inning1_runs[i]

            if team1_ids[i] == team2_id or team2_ids[i] == team2_id:
                matches_team2 += 1
                if team1_names[i] == toss_winners[i]:
                    if toss_decisions[i] == "bat":
                        bat_partnership_team1+=inning2_runs[i]/(inning2_wickets[i]) if inning2_wickets[i]!=0 else inning2_runs[i]
                        bat_partnership_team2+=inning1_runs[i]/(inning1_wickets[i]) if inning1_wickets[i]!=0 else inning1_runs[i]
                    else:
                        bat_partnership_team1+=inning1_runs[i]/(inning1_wickets[i]) if inning1_wickets[i]!=0 else inning1_runs[i]
                        bat_partnership_team2+=inning2_runs[i]/(inning2_wickets[i]) if inning2_wickets[i]!=0 else inning2_runs[i]
        
    
    bat_partnership_avg_team1=bat_partnership_team1 / matches_team1 if matches_team1 != 0 else 0
    bat_partnership_avg_team2=bat_partnership_team2 / matches_team2 if matches_team2 != 0 else 0

    return [bat_partnership_avg_team1,  bat_partnership_avg_team2]

In [39]:
from datetime import datetime

def highest_runs_scored(match_dt, team1_roster_ids, team2_roster_ids):
    # Convert the input match date to a datetime object
    match_dt = datetime.strptime(match_dt, '%Y-%m-%d')

    # Initialize variables to store highest runs
    highest_runs_team1 = 0
    highest_runs_team2 = 0
    highest_scorer_team1 = None
    highest_scorer_team2 = None

    # Convert roster IDs from strings to lists of integers
    team1_players = list(map(float, team1_roster_ids.split(':')))
    team2_players = list(map(float, team2_roster_ids.split(':')))


    # Iterate through each batsman's innings
    for i in range(len(batsman_match_ids)):
        match_dt_i = datetime.strptime(batsman_match_dates[i], '%Y-%m-%d')
        # print(match_dt_i, match_dt)
        if match_dt_i < match_dt:
            # print("inside match date")
            # Check if the batsman belongs to team1
            if batsman_ids[i] in team1_players:
                if runs[i] > highest_runs_team1:
                    highest_runs_team1 = runs[i]
                    highest_scorer_team1 = batsmen[i]
            
            # Check if the batsman belongs to team2
            if batsman_ids[i] in team2_players:
                if runs[i] > highest_runs_team2:
                    highest_runs_team2 = runs[i]
                    highest_scorer_team2 = batsmen[i]

    return [(highest_scorer_team1, highest_runs_team1),(highest_scorer_team2, highest_runs_team2)]

# Example usage:
# Assuming batsman_level is a pandas DataFrame containing the relevant data

# Load data (you would typically do this outside the function)
batsman_match_ids = batting_level['match id'].tolist()
batsman_ids = batting_level['batsman_id'].tolist()
batsmen = batting_level['batsman'].tolist()
runs = batting_level['runs'].tolist()
batsman_match_dates = batting_level['match_dt'].tolist()

# Call the function
result = highest_runs_scored('2022-06-03', '2089079.0:6282856.0:4640824.0:1798705.0:1613919.0:2111549.0:3573723.0:2022957.0:7672657.0:3901099.0:3231437.0', '87191.0:2958164.0:7787772.0:2083409.0:1707621.0:3620756.0:3373138.0:7687315.0:6399210.0:4473993.0:7732934.0')
print(f"Team 1 Highest Scorer: {result[0][0]}, Runs: {result[0][1]}")
print(f"Team 2 Highest Scorer: {result[1][0]}, Runs: {result[1][1]}")

Team 1 Highest Scorer: JJ Ry, Runs: 116
Team 2 Highest Scorer: DJ Bd, Runs: 88


In [40]:
def batting_strength(bat_avg, bat_strike_rate,  runs_scored, partnership, HIRS):
    return (0.4*bat_avg + 0.6*bat_strike_rate)*0.2 + 0.4*(runs_scored) + 0.3*(partnership) + 0.1*HIRS

In [41]:
train_matches = train_level["match id"]
bat_strength_team1_list = []
bat_strength_team2_list = []

bat_stk_rate_team1_list = []
bat_stk_rate_team2_list = []

for index, row in train_level.iterrows():
    team1_id = row['team1_id']
    team2_id = row['team2_id']
    match_dt = row['match_dt']
    team1_roster_ids = row['team1_roster_ids']
    team2_roster_ids = row['team2_roster_ids']
    
    bat_avg_team1, bat_avg_team2, bat_stk_rate_team1, bat_stk_rate_team2, runs_scored_team1, runs_scored_team2 = bat_avg(team1_id, team2_id, match_dt)
    partnership_team1, partnership_team2 = partnership(team1_id, team2_id, match_dt)
    result = highest_runs_scored(match_dt, team1_roster_ids, team2_roster_ids)
    hirs_team1 = result[0][1]
    hirs_team2 = result[1][1]

    bat_strength_team1 = batting_strength(bat_avg_team1, bat_stk_rate_team1, runs_scored_team1, partnership_team1, hirs_team1)
    bat_strength_team2 = batting_strength(bat_avg_team2, bat_stk_rate_team2, runs_scored_team2, partnership_team2, hirs_team2)

    bat_strength_team1_list.append(bat_strength_team1)
    bat_strength_team2_list.append(bat_strength_team2)

    bat_stk_rate_team1_list.append(bat_stk_rate_team1)
    bat_stk_rate_team2_list.append(bat_stk_rate_team2)

# Add the calculated batting strengths to the DataFrame
train_level['bat_strength_team1'] = bat_strength_team1_list
train_level['bat_strength_team2'] = bat_strength_team2_list

#External conditions
train_level['series_type'] = match_level['series_type']


train_level.to_csv('updated_train.csv', index = False)

In [42]:
test_matches = test_level["match id"]
bat_strength_team1_list = []
bat_strength_team2_list = []
bat_stk_rate_team1_list_test = []
bat_stk_rate_team2_list_test = []

for index, row in test_level.iterrows():
    team1_id = row['team1_id']
    team2_id = row['team2_id']
    match_dt = row['match_dt']
    team1_roster_ids = row['team1_roster_ids']
    team2_roster_ids = row['team2_roster_ids']
    
    
    bat_avg_team1, bat_avg_team2, bat_stk_rate_team1, bat_stk_rate_team2, runs_scored_team1, runs_scored_team2 = bat_avg(team1_id, team2_id, match_dt)
    partnership_team1, partnership_team2 = partnership(team1_id, team2_id, match_dt)
    result = highest_runs_scored(match_dt, team1_roster_ids, team2_roster_ids)
    hirs_team1 = result[0][1]
    hirs_team2 = result[1][1]

    bat_strength_team1 = batting_strength(bat_avg_team1, bat_stk_rate_team1, runs_scored_team1, partnership_team1, hirs_team1)
    bat_strength_team2 = batting_strength(bat_avg_team2, bat_stk_rate_team2, runs_scored_team2, partnership_team2, hirs_team2)

    bat_strength_team1_list.append(bat_strength_team1)
    bat_strength_team2_list.append(bat_strength_team2)

    bat_stk_rate_team1_list_test.append(bat_stk_rate_team1)
    bat_stk_rate_team2_list_test.append(bat_stk_rate_team2)

# Add the calculated batting strengths to the DataFrame
test_level['bat_strength_team1'] = bat_strength_team1_list
test_level['bat_strength_team2'] = bat_strength_team2_list

#External conditions
test_level['series_type'] = match_level['series_type']

test_level.to_csv('updated_test.csv', index = False)

In [43]:
epsilon = 1e-3

In [44]:
train_level['bat_strike_rate_team1'] = bat_stk_rate_team1_list
train_level['bat_strike_rate_team2'] = bat_stk_rate_team2_list
train_level['bat_strike_rate_team'] = train_level['bat_strike_rate_team1']/(train_level['bat_strike_rate_team2'] + 1)
# train_level['bat_strike_rate_team'] = (
#     (train_level['bat_strike_rate_team1'] - train_level['bat_strike_rate_team2']) / 
#     (train_level['bat_strike_rate_team1'] + train_level['bat_strike_rate_team2'] + epsilon)
# )

train_level.to_csv('updated_train.csv', index = False)


test_level['bat_strike_rate_team1'] = bat_stk_rate_team1_list_test
test_level['bat_strike_rate_team2'] = bat_stk_rate_team2_list_test
test_level['bat_strike_rate_team'] = test_level['bat_strike_rate_team1']/(test_level['bat_strike_rate_team2'] + 1)

# test_level['bat_strike_rate_team'] = (
#     (test_level['bat_strike_rate_team1'] - test_level['bat_strike_rate_team2']) / 
#     (test_level['bat_strike_rate_team1'] + test_level['bat_strike_rate_team2'] + epsilon)
# )

test_level.to_csv('updated_test.csv', index = False)


### Bowling performance

In [45]:
bowler_wides=bowler_level['wides'].tolist()
bowler_noballs=bowler_level['noballs'].tolist()
bowler_dots=bowler_level['dots'].tolist()
bowler_match_ids = bowler_level['match id'].tolist()
bowler_ids = bowler_level['bowler_id'].tolist()

In [46]:

def bowling_discipline(team1_id, team2_id, match_dt, team1_roster_ids, team2_roster_ids):
    # Convert the input match date to a datetime object
    match_dt = datetime.strptime(match_dt, '%Y-%m-%d')

    # Initialize the counters for extra runs and matches played
    bowler_extraruns_team1 = 0
    bowler_extraruns_team2 = 0
    bowler_dots_team1=0
    bowler_dots_team2=0
    matches_team1 = 0
    matches_team2 = 0

    # Convert roster IDs from strings to lists of integers
    team1_players = list(map(float, team1_roster_ids.split(':')))
    team2_players = list(map(float, team2_roster_ids.split(':')))

    # team1_players =  float(team1_roster_ids.split(':'))
    # team2_players = float(team2_roster_ids.split(':'))

    # Iterate through each match to calculate the extra runs conceded by each team
    for i in range(len(match_ids)):
        match_dt_i = datetime.strptime(match_date[i], '%Y-%m-%d')
        if match_dt_i < match_dt:
            if team1_ids[i] == team1_id or team2_ids[i] == team1_id:
                matches_team1 += 1
                
                for j in range(len(bowler_match_ids)):
                    if bowler_match_ids[j] == match_ids[i] and bowler_ids[j] in team1_players:
                        bowler_extraruns_team1 += bowler_wides[j] + bowler_noballs[j]
                        bowler_dots_team1+=bowler_dots[j]
            
            if team1_ids[i] == team2_id or team2_ids[i] == team2_id:
                matches_team2 += 1
                
                for j in range(len(bowler_match_ids)):
                    if bowler_match_ids[j] == match_ids[i] and bowler_ids[j] in team2_players:
                        bowler_extraruns_team2 += bowler_wides[j] + bowler_noballs[j]
                        bowler_dots_team2+=bowler_dots[j]

    # Calculate the average extra runs conceded per match by each team
    avg_extraruns_team1 = bowler_extraruns_team1 / matches_team1 if matches_team1 > 0 else 0
    avg_extraruns_team2 = bowler_extraruns_team2 / matches_team2 if matches_team2 > 0 else 0
    avg_dots_team1 = bowler_dots_team1 / matches_team1 if matches_team1 > 0 else 0
    avg_dots_team2 = bowler_dots_team2 / matches_team2 if matches_team2 > 0 else 0

    bowler_discipline_team1=   -1*avg_extraruns_team1 + 0.6*avg_dots_team1
    bowler_discipline_team2=   -1*avg_extraruns_team2 + 0.6*avg_dots_team2

    return [bowler_discipline_team1,bowler_discipline_team2]

# Example usage:
# Assuming bowler_level and match_level are pandas DataFrames containing the relevant data

In [47]:
def wicketClassifier(wickets):
    """
    this function is used to calculate number of 30s, 50s and 100s.
    returns -> num30,num50,num100
    """
    num5 = 0

    if wickets>=5:
        num5 += 1
   
    return 0.4*num5 + 0.6*wickets

In [48]:
def bowling_avg(team1_id, team2_id, match_dt):
    # # Find the index of the input match ID
    # match_index = match_ids.index(match_id_input)
    # team1_id = team1_ids[match_index]
    # team2_id = team2_ids[match_index]
    # match_dt = match_date[match_index]
    match_dt = datetime.strptime(match_dt, '%Y-%m-%d')

    # Initialize variables to calculate batting averages
    total_runs_team1 = 0
    total_runs_team2 = 0

    matches_team1 = 0
    matches_team2 = 0

    strike_rate_sum_team1 = 0
    strike_rate_sum_team2 = 0

    wickets_team1 = 0
    wickets_team2 = 0

    economy_team1=0
    economy_team2=0

    # Iterate through all matches to calculate batting averages
    for i in range(len(match_ids)):
        if match_dt > datetime.strptime(match_date[i], '%Y-%m-%d'):
            if team1_ids[i] == team1_id or team2_ids[i] == team1_id:
                matches_team1 += 1
                if toss_winners[i] == team1_names[i]:
                    if toss_decisions[i] == 'field':

                        total_runs_team1 += inning1_runs[i]/inning1_wickets[i] if inning1_wickets[i] !=0 else inning1_runs[i]
                        strike_rate_sum_team1 +=  inning1_balls[i]/inning1_wickets[i] if inning1_wickets[i] !=0 else inning1_balls[i]
                        economy_team1+= inning1_runs[i]/inning1_balls[i]
                        wickets_team1 += wicketClassifier(inning1_wickets[i])
                    else:
                        
                        total_runs_team1 += inning2_runs[i]/inning2_wickets[i] if inning2_wickets[i] !=0 else inning2_runs[i]
                        strike_rate_sum_team1 += inning2_balls[i]/inning2_wickets[i] if inning2_wickets[i] !=0 else inning2_balls[i]
                        economy_team1+= inning2_runs[i]/inning2_balls[i]
                        wickets_team1 += wicketClassifier(inning2_wickets[i])
                
                else:
                    if toss_decisions[i] == 'field':
                        total_runs_team1 += inning2_runs[i]/inning2_wickets[i] if inning2_wickets[i] !=0 else inning2_runs[i]
                        strike_rate_sum_team1 += inning2_balls[i]/inning2_wickets[i] if inning2_wickets[i] !=0 else inning2_balls[i]
                        economy_team1+= inning2_runs[i]/inning2_balls[i]
                        wickets_team1 += wicketClassifier(inning2_wickets[i])

                    else:
                        total_runs_team1 += inning1_runs[i]/inning1_wickets[i] if inning1_wickets[i] !=0 else inning1_runs[i]
                        strike_rate_sum_team1 += inning1_balls[i]/inning1_wickets[i] if inning1_wickets[i] !=0 else inning1_balls[i]
                        economy_team1+= inning1_runs[i]/inning1_balls[i]
                        wickets_team1 += wicketClassifier(inning1_wickets[i])
            
            if team1_ids[i] == team2_id or team2_ids[i] == team2_id:
                matches_team2 += 1
                if toss_winners[i] == team2_names[i]:
                    if toss_decisions[i] == 'field':
                        total_runs_team2 += inning1_runs[i]/inning1_wickets[i] if inning1_wickets[i] !=0 else inning1_runs[i]
                        strike_rate_sum_team2 += inning1_balls[i]/inning1_wickets[i] if inning1_wickets[i] !=0 else inning1_balls[i]
                        economy_team2+= inning1_runs[i]/inning1_balls[i]
                        wickets_team2 += wicketClassifier(inning1_wickets[i])
                    else:
                        total_runs_team2 += inning2_runs[i]/inning2_wickets[i] if inning2_wickets[i] !=0 else inning2_runs[i]
                        strike_rate_sum_team2 += inning2_balls[i]/inning2_wickets[i] if inning2_wickets[i] !=0 else inning2_balls[i]
                        economy_team2+= inning1_runs[i]/inning1_balls[i]
                        wickets_team2 += wicketClassifier(inning2_wickets[i])
                else:
                    if toss_decisions[i] == 'field':
                        total_runs_team2 += inning2_runs[i]/inning2_wickets[i] if inning2_wickets[i] !=0 else inning2_runs[i]
                        strike_rate_sum_team2 += inning2_balls[i]/inning2_wickets[i] if inning2_wickets[i] !=0 else inning2_balls[i]
                        economy_team2+= inning1_runs[i]/inning1_balls[i]
                        wickets_team2 += wicketClassifier(inning2_wickets[i])
                    else:
                        total_runs_team2 += inning1_runs[i]/inning1_wickets[i] if inning1_wickets[i] !=0 else inning1_runs[i]
                        strike_rate_sum_team2 += inning1_balls[i]/inning1_wickets[i] if inning1_wickets[i] !=0 else inning1_balls[i]
                        economy_team2+= inning1_runs[i]/inning1_balls[i]
                        wickets_team2 += wicketClassifier(inning1_wickets[i])

    
    bowling_avg_team1 = total_runs_team1 / matches_team1 if matches_team1 != 0 else 0
    bowling_avg_team2 = total_runs_team2 / matches_team2 if matches_team2 != 0 else 0
    strike_rate_avg_team1 = strike_rate_sum_team1/ matches_team1 if matches_team1 != 0 else 0
    strike_rate_avg_team2 = strike_rate_sum_team2/ matches_team2 if matches_team2 != 0 else 0
    economy_team1_avg=economy_team1/matches_team1  if matches_team1 != 0 else 0
    economy_team2_avg=economy_team2/matches_team2  if matches_team2 != 0 else 0

    return [bowling_avg_team1, bowling_avg_team2, strike_rate_avg_team1, strike_rate_avg_team2, wickets_team1, wickets_team2,economy_team1_avg*6,economy_team2_avg*6]


# match_id_input = 9433696

bowl_avg_team1, bowl_avg_team2, strike_rate_team1, strike_rate_team2 , wickets_team1, wickets_team2, economy_team1,economy_team2 = bowling_avg(12634, 14860, "2022-10-18")

print(f"Bowling average for Team 1: {bowl_avg_team1}")
print(f"Bowling average for Team 2: {bowl_avg_team2}")

print(f"Strike rate team 1: {strike_rate_team1}")
print(f"Strike rate team 2: {strike_rate_team2}")

print(f"wickets team 1: {wickets_team1}")
print(f"wickets team 2: {wickets_team2}")

print(economy_team2)
print(economy_team1)

Bowling average for Team 1: 40.11805555555556
Bowling average for Team 2: 34.911630036630044
Strike rate team 1: 32.43361111111111
Strike rate team 2: 24.683699633699632
wickets team 1: 37.6
wickets team 2: 45.8
8.508322883179204
6.8267192779253


In [49]:
def bowling_strength(bowl_avg, bowl_strike_rate, economy, wickets,discipline):
    return (0.3*bowl_avg + 0.7*bowl_strike_rate + 0*economy)*0.33 + 0.33*(wickets) + 0.33*(discipline)

### Train data

In [50]:
train_matches = train_level["match id"]
bowl_strength_team1_list = []
bowl_strength_team2_list = []
team1_economy_list = []
team2_economy_list = []

for index, row in train_level.iterrows():
    team1_id = row['team1_id']
    team2_id = row['team2_id']
    match_dt = row['match_dt']
    team1_roster_id = row['team1_roster_ids']
    team2_roster_id = row['team2_roster_ids']
    
    bowl_avg_team1, bowl_avg_team2, bowl_stk_rate_team1, bowl_stk_rate_team2, wicket_scored_team1, wicket_scored_team2, economy_team1, economy_team2 = bowling_avg(team1_id, team2_id, match_dt)
    discipline_team1, discipline_team2 = bowling_discipline(team1_id, team2_id, match_dt, team1_roster_id, team2_roster_id)

    bowl_strength_team1 = bowling_strength(bowl_avg_team1, bowl_stk_rate_team1,economy_team1, wicket_scored_team1, discipline_team1)
    bowl_strength_team2 = bowling_strength(bowl_avg_team2, bowl_stk_rate_team2,economy_team2, wicket_scored_team2, discipline_team2)

    bowl_strength_team1_list.append(bowl_strength_team1)
    bowl_strength_team2_list.append(bowl_strength_team2)

    team1_economy_list.append(economy_team1)
    team2_economy_list.append(economy_team2)

# Add the calculated batting strengths to the DataFrame
train_level['bowl_strength_team1'] = bowl_strength_team1_list
train_level['bowl_strength_team2'] = bowl_strength_team2_list
train_level['economy_team1'] = team1_economy_list
train_level['economy_team2'] = team2_economy_list

train_level.to_csv('updated_train.csv', index = False)

### Test data

In [51]:

bowl_strength_team1_list = []
bowl_strength_team2_list = []
team1_economy_list = []
team2_economy_list = []

for index, row in test_level.iterrows():
    team1_id = row['team1_id']
    team2_id = row['team2_id']
    match_dt = row['match_dt']
    team1_roster_id = row['team1_roster_ids']
    team2_roster_id = row['team2_roster_ids']
    
    bowl_avg_team1, bowl_avg_team2, bowl_stk_rate_team1, bowl_stk_rate_team2, wicket_scored_team1, wicket_scored_team2, economy_team1, economy_team2 = bowling_avg(team1_id, team2_id, match_dt)
    discipline_team1, discipline_team2 = bowling_discipline(team1_id, team2_id, match_dt, team1_roster_id, team2_roster_id)

    bowl_strength_team1 = bowling_strength(bowl_avg_team1, bowl_stk_rate_team1,economy_team1, wicket_scored_team1, discipline_team1)
    bowl_strength_team2 = bowling_strength(bowl_avg_team2, bowl_stk_rate_team2,economy_team2, wicket_scored_team2, discipline_team2)

    bowl_strength_team1_list.append(bowl_strength_team1)
    bowl_strength_team2_list.append(bowl_strength_team2)

    team1_economy_list.append(economy_team1)
    team2_economy_list.append(economy_team2)


# Add the calculated batting strengths to the DataFrame
test_level['bowl_strength_team1'] = bowl_strength_team1_list
test_level['bowl_strength_team2'] = bowl_strength_team2_list
test_level['economy_team1'] = team1_economy_list
test_level['economy_team2'] = team2_economy_list


test_level.to_csv('updated_test.csv', index = False)

In [52]:
train_level['team1_strength'] = 0.5*train_level['bat_strength_team1'] + 0.5*train_level['bowl_strength_team1']
train_level['team2_strength'] = 0.5*train_level['bat_strength_team2'] + 0.5*train_level['bowl_strength_team2']
train_level['team_strength'] = train_level['team1_strength']/(train_level['team2_strength'] +1)
train_level['economy_team'] = train_level['economy_team1']/(train_level['economy_team2']+1)
# train_level['economy_team'] = (
#     (train_level['economy_team1'] - train_level['economy_team2']) / 
#     (train_level['economy_team1'] + train_level['economy_team2'] + epsilon)
# )

test_level['team1_strength'] = 0.5*test_level['bat_strength_team1'] + 0.5*test_level['bowl_strength_team1']
test_level['team2_strength'] = 0.5*test_level['bat_strength_team2'] + 0.5*test_level['bowl_strength_team2']
test_level['team_strength'] = test_level['team1_strength']/(test_level['team2_strength'] + 1)
test_level['economy_team'] = test_level['economy_team1']/(test_level['economy_team2']+1)
# test_level['economy_team'] = (
#     (test_level['economy_team1'] - test_level['economy_team2']) / 
#     (test_level['economy_team1'] + test_level['economy_team2'] + epsilon)
# )

train_level.to_csv('updated_train.csv', index = False)
test_level.to_csv('updated_test.csv', index = False)

### Ground features

In [53]:
ground_ids = match_level['ground_id'].tolist()
winner_ids = match_level['winner_id'].tolist()

In [54]:
def win_p_ground(team1_id,input_ground_id,match_dt):
    
    wins=0
    matches_team1=0
    match_dt = datetime.strptime(match_dt, '%Y-%m-%d')


    for i in range(len(match_ids)):
        if ground_ids[i]==input_ground_id and match_dt > datetime.strptime(match_date[i], '%Y-%m-%d'):
            if team1_ids[i] == team1_id or team2_ids[i] == team1_id:
                matches_team1 += 1
                if team1_ids[i] == winner_ids[i]:
                    wins+=1
                
    return wins / matches_team1 if matches_team1 != 0 else 0

In [55]:
epsilon = 1e-3

In [56]:
ground_ids = match_level['ground_id'].tolist()
winner_ids = match_level['winner_id'].tolist()

def ground_toss_vs_wins(team1_id, match_dt, input_ground_id):
    """
    returns ratio of freq of bat first wins and bowl first wins
    """
    match_dt = datetime.strptime(match_dt, '%Y-%m-%d')


    bat_first_win = 0
    bowl_first_win = 0
    

    # Iterate through all matches to calculate batting averages
    for i in range(len(match_ids)):
        if match_dt > datetime.strptime(match_date[i], '%Y-%m-%d') and ground_ids[i] == input_ground_id :


            if team1_ids[i] == team1_id or team2_ids[i] == team1_id:
                if toss_winners[i] == team1_names[i]:
                    if toss_decisions[i] == 'bat':
                        if (winner_ids[i] == team1_id):
                            bat_first_win+=1
                    else:
                        if (winner_ids[i] == team1_id):
                            bowl_first_win+=1
                
                else:
                    if toss_decisions[i] == 'bat':
                        if winner_ids[i] == team1_id:
                            bowl_first_win+=1
                        
                    else:
                        if winner_ids[i] == team1_id:
                            bat_first_win+=1
    
    return ((bat_first_win - bowl_first_win) / (bat_first_win + bowl_first_win + epsilon))

In [57]:
# ground_toss_vs_wins(team1_id = ground_id = 23841)

In [58]:
wins_grd_team1_list=[]
wins_grd_team2_list=[]

ground_toss_vs_wins_team1_list = []
ground_toss_vs_wins_team2_list = []

for index, row in train_level.iterrows():
    team1_id = row['team1_id']
    team2_id = row['team2_id']
    match_dt = row['match_dt']
    team1_roster_id = row['team1_roster_ids']
    team2_roster_id = row['team2_roster_ids']
    ground_id = row['ground_id']

    wins_grd_team1 = win_p_ground(team1_id, ground_id, match_dt)
    wins_grd_team2 = win_p_ground(team2_id, ground_id, match_dt)

    wins_toss_vs_wins_team1 = ground_toss_vs_wins(team1_id, match_dt, ground_id)
    wins_toss_vs_wins_team2 = ground_toss_vs_wins(team2_id, match_dt, ground_id)


    wins_grd_team1_list.append(wins_grd_team1)
    wins_grd_team2_list.append(wins_grd_team2)

    ground_toss_vs_wins_team1_list.append(wins_toss_vs_wins_team1)
    ground_toss_vs_wins_team2_list.append(wins_toss_vs_wins_team2)

train_level['wins_ground_team1'] = wins_grd_team1_list
train_level['wins_ground_team2'] = wins_grd_team2_list
train_level['wins_ground_team'] = train_level['wins_ground_team1']/(train_level['wins_ground_team2'] + 1)

train_level['ground_toss_vs_wins_team1'] = ground_toss_vs_wins_team1_list
train_level['ground_toss_vs_wins_team2'] = ground_toss_vs_wins_team2_list

# train_level['ground_toss_vs_wins_team'] = np.where(
#     train_level['ground_toss_vs_wins_team2'] != 0,
#     train_level['ground_toss_vs_wins_team1'] / train_level['ground_toss_vs_wins_team2'],
#     train_level['ground_toss_vs_wins_team1']
# )

# small constant to avoid division by zero
# train_level['ground_toss_vs_wins_team_ratio'] = train_level['ground_toss_vs_wins_team1'] / (train_level['ground_toss_vs_wins_team2'] + epsilon)

train_level['ground_toss_vs_wins_team_norm_diff'] = ((train_level['ground_toss_vs_wins_team1'] - train_level['ground_toss_vs_wins_team2']) / (train_level['ground_toss_vs_wins_team1'] + train_level['ground_toss_vs_wins_team2'] + epsilon))

train_level.to_csv('updated_train.csv', index = False)

In [59]:
wins_grd_team1_list=[]
wins_grd_team2_list=[]

ground_toss_vs_wins_team1_list = []
ground_toss_vs_wins_team2_list = []

for index, row in test_level.iterrows():
    team1_id = row['team1_id']
    team2_id = row['team2_id']
    match_dt = row['match_dt']
    team1_roster_id = row['team1_roster_ids']
    team2_roster_id = row['team2_roster_ids']
    ground_id = row['ground_id']

    wins_grd_team1 = win_p_ground(team1_id, ground_id, match_dt)
    wins_grd_team2 = win_p_ground(team2_id, ground_id, match_dt)

    wins_toss_vs_wins_team1 = ground_toss_vs_wins(team1_id, match_dt, ground_id)
    wins_toss_vs_wins_team2 = ground_toss_vs_wins(team2_id, match_dt, ground_id)


    wins_grd_team1_list.append(wins_grd_team1)
    wins_grd_team2_list.append(wins_grd_team2)

    ground_toss_vs_wins_team1_list.append(wins_toss_vs_wins_team1)
    ground_toss_vs_wins_team2_list.append(wins_toss_vs_wins_team2)

test_level['wins_ground_team1'] = wins_grd_team1_list
test_level['wins_ground_team2'] = wins_grd_team2_list
test_level['wins_ground_team'] = test_level['wins_ground_team1']/(test_level['wins_ground_team2'] + 1)

test_level['ground_toss_vs_wins_team1'] = ground_toss_vs_wins_team1_list
test_level['ground_toss_vs_wins_team2'] = ground_toss_vs_wins_team2_list
# test_level['ground_toss_vs_wins_team'] = np.where(
#     test_level['ground_toss_vs_wins_team2'] != 0,
#     test_level['ground_toss_vs_wins_team1'] / test_level['ground_toss_vs_wins_team2'],
#     test_level['ground_toss_vs_wins_team1']
# )

# test_level['ground_toss_vs_wins_team_ratio'] = test_level['ground_toss_vs_wins_team1'] / (test_level['ground_toss_vs_wins_team2'] + epsilon)

test_level['ground_toss_vs_wins_team_norm_diff'] = ((test_level['ground_toss_vs_wins_team1'] - test_level['ground_toss_vs_wins_team2']) / (test_level['ground_toss_vs_wins_team1'] + test_level['ground_toss_vs_wins_team2'] + epsilon))

test_level.to_csv('updated_test.csv', index = False)